In [2]:
#install.packages('bigsnpr')
library(bigsnpr)

In [3]:
system("dx pwd",intern=TRUE)   # View current directory in the DNAnexus project

# Create a test file and read it, as it were from bash terminal
# system("dx download PRS_reporting:/Input/geno/chr1.*",intern=TRUE)
# snp_readBed("chr1.bed")
# snp_readBed("PRS_reporting:/Input/geno/chr1.bed")

system("dx ls",intern=TRUE)   # View current directory in the DNAnexus project

[1] "PRS_reporting:/Scripts"

[1] "Filtering VCF"                           
[2] "PRS_computing.ipynb"                     
[3] "PRSice"                                  
[4] "PRSice (Copy: Jun 3rd 2021 9:53am)"      
[5] "Untitled Workflow - Jun 3rd 2021 10:32pm"
[6] "Untitled Workflow - Jun 3rd 2021 4:28pm" 
[7] "Untitled_2021-6-3T14-52-18.ipynb"        
[8] "vcf2plink"                               
[9] "VCFOnwards"

In [4]:
# Attach the "bigSNP" object in R session
obj.bigSNP <- snp_attach("chr1.rds")
# See how the file looks like
str(obj.bigSNP, max.level = 2, strict.width = "cut")

List of 3
 $ genotypes:Reference class 'FBM.code256' [package "bigstatsr"] with 16 fields
  ..and 26 methods, of which 12 are  possibly relevant:
  ..  add_columns, as.FBM, bm, bm.desc, check_dimensions,
  ..  check_write_permissions, copy#envRefClass, initialize, initialize#FBM,
  ..  save, show#envRefClass, show#FBM
 $ fam      :'data.frame':	100000 obs. of  6 variables:
  ..$ family.ID  : chr [1:100000] "sample_1_0" "sample_1_1" "sample_1_10" "sa"..
  ..$ sample.ID  : chr [1:100000] "sample_1_0" "sample_1_1" "sample_1_10" "sa"..
  ..$ paternal.ID: int [1:100000] 0 0 0 0 0 0 0 0 0 0 ...
  ..$ maternal.ID: int [1:100000] 0 0 0 0 0 0 0 0 0 0 ...
  ..$ sex        : int [1:100000] 2 1 1 1 1 1 2 1 2 2 ...
  ..$ affection  : int [1:100000] -9 -9 -9 -9 -9 -9 -9 -9 -9 -9 ...
 $ map      :'data.frame':	913287 obs. of  6 variables:
  ..$ chromosome  : int [1:913287] 1 1 1 1 1 1 1 1 1 1 ...
  ..$ marker.ID   : chr [1:913287] "chr1_12748_G_C" "chr1_12754_C_G" "chr1_13"..
  ..$ genetic.dist: int 

In [5]:
# Get aliases for useful slots
G   <- obj.bigSNP$genotypes
CHR <- obj.bigSNP$map$chromosome
POS <- obj.bigSNP$map$physical.pos
y   <- obj.bigSNP$fam$affection - 1
NCORES <- nb_cores()
# Check some counts for the 10 first variants
G <- G$copy(code=c(0,1,2,0, rep(NA,252)))
big_counts(G, ind.col = 1:10)

0,99954,99966,99992,99994,99617,99992,99970,99992,99990,99989
1,34,24,6,5,377,6,26,8,7,8
2,12,10,2,1,6,2,4,0,3,3
NA,0,0,0,0,0,0,0,0,0,0


In [6]:
# system("dx download PRS_reporting:/Input/gwas_catalog/trytrying_again_chr1.tsv.gz",intern=TRUE)

Warning message in system("dx download PRS_reporting:/Input/gwas_catalog/trytrying_again_chr1.tsv.gz", :
“running command 'dx download PRS_reporting:/Input/gwas_catalog/trytrying_again_chr1.tsv.gz' had status 1”


character(0)
attr(,"status")
[1] 1
attr(,"errmsg")
[1] "Resource temporarily unavailable"

In [7]:
sumstats <- read.table(gzfile("trytrying_again_chr1.tsv.gz"), stringsAsFactors=FALSE, header=TRUE)
str(sumstats)

'data.frame':	1537155 obs. of  7 variables:
 $ SNP: chr  "1:10177:A:AC" "rs145072688:10352:T:TA" "rs376342519:10616:CCGCCGTTGCAAAGGCGCGCCG:C" "1:11008:C:G" ...
 $ CHR: int  1 1 1 1 1 1 1 1 1 1 ...
 $ BP : int  10177 10352 10616 11008 11012 13110 13116 13118 13273 13289 ...
 $ A1 : chr  "AC" "TA" "C" "G" ...
 $ A2 : chr  "A" "T" "CCGCCGTTGCAAAGGCGCGCCG" "C" ...
 $ OR : num  -0.0949 -0.0898 0.3528 0.192 0.192 ...
 $ P  : num  0.0937 0.0957 0.1339 0.0519 0.0519 ...


In [8]:
names(sumstats) <- c("rsid", "chr", "pos", "a1", "a0", "beta", "p")
sumstats$p <- as.numeric(sumstats$p)
sumstats$pos <- as.numeric(sumstats$pos)
map <- obj.bigSNP$map[,-(2:3)]
head(map)
head(sumstats)
names(map) <- c("chr", "pos", "a0", "a1")
info_snp <- snp_match(sumstats, map, match.min.prop = 0)

,chromosome,physical.pos,allele1,allele2
,<int>,<int>,<chr>,<chr>
1,1,12748,C,G
2,1,12754,G,C
3,1,13367,C,G
4,1,13370,C,T
5,1,13380,G,C
6,1,13382,G,C


,rsid,chr,pos,a1,a0,beta,p
,<chr>,<int>,<dbl>,<chr>,<chr>,<dbl>,<dbl>
1,1:10177:A:AC,1,10177,AC,A,-0.09490,0.09372
2,rs145072688:10352:T:TA,1,10352,TA,T,-0.08981,0.09566
3,rs376342519:10616:CCGCCGTTGCAAAGGCGCGCCG:C,1,10616,C,CCGCCGTTGCAAAGGCGCGCCG,0.35280,0.13390
4,1:11008:C:G,1,11008,G,C,0.19200,0.05185
5,1:11012:C:G,1,11012,G,C,0.19200,0.05185
6,1:13110:G:A,1,13110,A,G,-0.35260,0.24370


1,537,155 variants to be matched.

778 ambiguous SNPs have been removed.

2,743 variants have been matched; 1,378 were flipped and 1,468 were reversed.



In [9]:
info_snp

chr,pos,a0,a1,rsid,beta,p,_NUM_ID_.ss,_NUM_ID_
<int>,<int>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<int>,<int>
1,13543,G,T,1:13543:T:G,0.039530,0.939400,11,19
1,69534,C,T,rs190717287:69534:T:C,0.545700,0.343900,100,92
1,138484,G,A,1:138484:A:G,6.239000,0.009516,195,138
1,138802,G,T,1:138802:T:G,-0.663700,0.816900,196,198
1,941284,A,G,rs3128116:941284:C:T,-0.027750,0.579600,2609,806
1,944063,T,C,rs28386817:944063:G:A,-0.593200,0.502400,2630,1108
1,945152,A,G,rs150148832:945152:T:C,-0.779000,0.759000,2642,1266
1,945612,T,C,rs3121565:945612:C:T,-0.063720,0.828800,2648,1318
1,946579,A,G,1:946579:G:A,0.732200,0.454700,2654,1456


In [10]:
beta <- rep(NA, ncol(G))
beta[info_snp$`_NUM_ID_`] <- info_snp$beta
lpval <- rep(NA, ncol(G))
lpval[info_snp$`_NUM_ID_`] <- -log10(info_snp$p)

In [11]:
# The clumping step might take some time to complete
all_keep <- snp_grid_clumping(G, CHR, POS, ind.row = 1:length(y),
                              lpS = lpval, exclude = which(is.na(lpval)),
                              ncores = NCORES)
attr(all_keep, "grid")

size,thr.r2,grp.num,thr.imp
<int>,<dbl>,<int>,<dbl>
5000,0.01,1,1
10000,0.01,1,1
20000,0.01,1,1
50000,0.01,1,1
1000,0.05,1,1
2000,0.05,1,1
4000,0.05,1,1
10000,0.05,1,1
500,0.10,1,1


In [13]:
multi_PRS <- snp_grid_PRS(G, all_keep, beta, lpval, ind.row = 1:length(y),
                          backingfile = "tmp-data/tmp", 
                          n_thr_lpS = 50, ncores = NCORES)
dim(multi_PRS)  ## 1400 C+T scores for 100000 individuals

[1] 100000   1400

In [15]:
ID <- obj.bigSNP$fam$family.ID
PRS_results <- data.frame(FID=ID, PID=ID)

In [16]:
prs_ <- multi_PRS[]
PRS_scores <- as.data.frame(prs_)

In [17]:
dim(PRS_results)
dim(PRS_scores)
final_output <- cbind(PRS_results, PRS_scores)
dim(final_output)
write.csv(final_output, 'PRS.csv', row.names=FALSE)

[1] 100000      2

[1] 100000   1400

[1] 100000   1402